In [1]:
# User churn problem from the rel-hm dataset
# on relbench: https://relbench.stanford.edu/datasets/rel-hm/#user-churn

# 1) https://demo.kurve.ai
# 2) create graph of rel-hm data
# 3) assign customer as parent node and transactions as the label node with depth 4


In [2]:
import pandas as pd
from torch_frame.utils import infer_df_stype
import catboost
from sklearn import metrics

In [3]:
train_path = 'https://kurve-customers.s3.amazonaws.com/4e1a245a-3065-4600-bb0e-a92e06ee835c/13/output/hm_user_churn_train'

In [4]:
df = pd.read_parquet(train_path)


In [5]:
df.shape

(1371980, 166)

In [6]:
[c for c in df.columns if 'label' in c]

['tran_article_id_label']

In [7]:
target = [c for c in df.columns if 'label' in c][0]

In [8]:
df[target]

0           2
1           3
2           2
3           4
4           8
           ..
1371975     4
1371976     1
1371977     1
1371978     1
1371979    10
Name: tran_article_id_label, Length: 1371980, dtype: Int64

In [10]:
len(df[df[target].isnull()])/len(df)

0.9456442513739267

In [11]:
# 94% churned

In [12]:

df[target] = df[target].apply(
    lambda x: 1 if x == 0 or pd.isnull(x) else 0)

In [13]:
df[target].sum()/len(df)

0.9456442513739267

In [14]:
# execute compute graph with
# cut date 9/14/2020 and load data

In [17]:
test = pd.read_parquet(train_path)
test.shape

(1371980, 166)

In [18]:
len(test[test[target].isnull()])/len(test)

0.9510605110861674

In [19]:
test[target] = test[target].apply(
    lambda x: 1 if x == 0 or pd.isnull(x) else 0)

In [20]:
test.head(2)

,cust_customer_id,cust_FN,cust_Active,cust_club_member_status,cust_fashion_news_frequency,cust_age,cust_postal_code,tran_t_dat_min,tran_t_dat_max,tran_customer_id_count,...,tran_3dv4_change,tran_4dv7_change,tran_7dv14_change,tran_14dv30_change,tran_30dv60_change,tran_60dv90_change,tran_90dv180_change,tran_180dv365_change,tran_365dv730_change,tran_article_id_label
0,1156056,NaN,NaN,ACTIVE,NONE,21.0,229ea73d251c531740206a82c082f2db870a03011b299c...,2019-10-22,2020-08-19,20,...,NaN,NaN,NaN,0.0,0.75,0.8,0.312500,0.8,1.0,0
1,1164440,1.0,1.0,ACTIVE,Regularly,54.0,62e88497d01a575306c27e988b8910fe0b51cd804c1856...,2019-10-21,2020-07-15,10,...,NaN,NaN,NaN,NaN,NaN,0.0,0.333333,0.6,1.0,0


In [23]:
stypes = infer_df_stype(test)
features = [
    k for k,v in stypes.items()
    if str(v) == 'numerical'
    and 'label' not in k
    and not k.startswith('cust_')
]

In [24]:
len(features)

123

In [25]:
features.append('cust_age')

In [26]:
features

['tran_customer_id_count',
 'tran_price_avg',
 'tran_price_sum',
 'tran_price_min',
 'tran_price_max',
 'arti_article_id_count_sum',
 'arti_product_code_avg_avg',
 'arti_product_code_avg_sum',
 'arti_product_code_avg_min',
 'arti_product_code_avg_max',
 'arti_product_code_sum_avg',
 'arti_product_code_sum_sum',
 'arti_product_code_sum_min',
 'arti_product_code_sum_max',
 'arti_product_code_min_avg',
 'arti_product_code_min_sum',
 'arti_product_code_min_min',
 'arti_product_code_min_max',
 'arti_product_code_max_avg',
 'arti_product_code_max_sum',
 'arti_product_code_max_min',
 'arti_product_code_max_max',
 'arti_product_type_no_avg_avg',
 'arti_product_type_no_avg_sum',
 'arti_product_type_no_avg_min',
 'arti_product_type_no_avg_max',
 'arti_product_type_no_sum_avg',
 'arti_product_type_no_sum_sum',
 'arti_product_type_no_sum_min',
 'arti_product_type_no_sum_max',
 'arti_product_type_no_min_avg',
 'arti_product_type_no_min_sum',
 'arti_product_type_no_min_min',
 'arti_product_type_no_m

In [27]:
import numpy as np
from sklearn.model_selection import StratifiedKFold, train_test_split
from sklearn.metrics import roc_auc_score
from catboost import CatBoostClassifier, Pool

# -------------------------------------------------
# 1. Split off the *final* test set (once!)
# -------------------------------------------------
X_train_full, X_test, y_train_full, y_test = train_test_split(
    df[features], df[target],
    test_size=0.20,          # 20 % held-out test
    stratify=df[target],
    random_state=42
)

# -------------------------------------------------
# 2. K-Fold CV on the remaining 80 %
# -------------------------------------------------
k = 3                                   # change to 10, etc.
skf = StratifiedKFold(n_splits=k, shuffle=True, random_state=42)

fold_aucs = []
test_preds = np.zeros(len(X_test))      # out-of-fold predictions on *test*
oof_preds = np.zeros(len(X_train_full)) # optional: OOF on training data

# CatBoost pools (optional but faster)
train_pool = Pool(X_train_full, y_train_full)  # , cat_features=cat_features)

for fold, (idx_tr, idx_va) in enumerate(skf.split(X_train_full, y_train_full), 1):
    print(f"\n=== Fold {fold} ===")
    
    X_tr, X_va = X_train_full.iloc[idx_tr], X_train_full.iloc[idx_va]
    y_tr, y_va = y_train_full.iloc[idx_tr], y_train_full.iloc[idx_va]

    # -----------------------------------------------------------------
    # 3. Fit on the *training* split of this fold
    # -----------------------------------------------------------------
    mdl = catboost.CatBoostClassifier(
        # objective & metrics
        loss_function="Logloss",
        eval_metric="AUC",
        custom_metric=["AUC", "PRAUC", "F1", "Recall", "Precision", "Logloss"],
        use_best_model=True,
    
        # capacity vs regularization
        iterations=2000,
        learning_rate=0.05,
        depth=5,
        l2_leaf_reg=10,
    
        # randomness / bagging / feature subsampling
        bootstrap_type="Bayesian",
        bagging_temperature=0.75,
        rsm=0.8,
        random_strength=0.5,
    
        # class imbalance handling
        auto_class_weights="Balanced",
        #class_weights=[0.6,0.4],
    
        # borders / leaves
        feature_border_type="GreedyLogSum",
        min_data_in_leaf=20,
        boosting_type="Plain",
    
        # early stopping
        od_type="Iter",
        od_wait=150,
    
        verbose=200
    )
    mdl.fit(
        X_tr, y_tr,
        eval_set=(X_va, y_va),
        use_best_model=False,
        verbose=False
    )

    # -------------------------------------------------
    # 4. Validation AUC for this fold
    # -------------------------------------------------
    val_pred = mdl.predict_proba(X_va)[:, 1]
    val_auc  = roc_auc_score(y_va, val_pred)
    fold_aucs.append(val_auc)
    print(f"Fold {fold} validation AUC : {val_auc:.4f}")

    # -------------------------------------------------
    # 5. Accumulate predictions on the *final* test set
    # -------------------------------------------------
    test_preds += mdl.predict_proba(X_test)[:, 1] / k

    # (optional) OOF on training data
    oof_preds[idx_va] = val_pred

# -------------------------------------------------
# 6. Final metrics
# -------------------------------------------------
print("\n=== CV Summary ===")
print(f"Mean CV AUC : {np.mean(fold_aucs):.4f} ± {np.std(fold_aucs):.4f}")
print(f"Folds AUC   : {[f'{a:.4f}' for a in fold_aucs]}")

test_auc = roc_auc_score(y_test, test_preds)
print(f"\nFinal test AUC (averaged over {k} folds): {test_auc:.4f}")

# -------------------------------------------------
# 7. (Optional) Refit on the *whole* train_full for deployment
# -------------------------------------------------
final_mdl = catboost.CatBoostClassifier(
        # objective & metrics
        loss_function="Logloss",
        eval_metric="AUC",
        custom_metric=["AUC", "PRAUC", "F1", "Recall", "Precision", "Logloss"],
        #use_best_model=True,
    
        # capacity vs regularization
        #iterations=2000,
        iterations=int(mdl.best_iteration_ * 1.1),  # a bit more than best
        learning_rate=0.05,
        depth=5,
        l2_leaf_reg=10,
    
        # randomness / bagging / feature subsampling
        bootstrap_type="Bayesian",
        bagging_temperature=0.75,
        rsm=0.8,
        random_strength=0.5,
    
        # class imbalance handling
        auto_class_weights="Balanced",
        #class_weights=[0.6,0.4],
    
        # borders / leaves
        feature_border_type="GreedyLogSum",
        min_data_in_leaf=20,
        boosting_type="Plain",
    
        # early stopping
        od_type="Iter",
        od_wait=150,
        verbose=200
    )
final_mdl.fit(X_train_full, y_train_full)


=== Fold 1 ===
Fold 1 validation AUC : 0.7729

=== Fold 2 ===
Fold 2 validation AUC : 0.7753

=== Fold 3 ===
Fold 3 validation AUC : 0.7717

=== CV Summary ===
Mean CV AUC : 0.7733 ± 0.0015
Folds AUC   : ['0.7729', '0.7753', '0.7717']

Final test AUC (averaged over 3 folds): 0.7697
0:	total: 82.6ms	remaining: 46.6s
200:	total: 32.6s	remaining: 59.1s
400:	total: 1m 7s	remaining: 27.6s
564:	total: 1m 35s	remaining: 0us


In [28]:
test['prob'] = final_mdl.predict_proba(test[features])[:,1]

In [29]:
metrics.roc_auc_score(test[target], test['prob'])

0.7719574766374032